## Author: Blaine Hill

In this notebook, we train a score matching model as described by [Yang Song](https://yang-song.net/blog/2021/score/)


$$s_\theta(x) = \nabla_x \log p_\theta(x) = -\nabla_x f_\theta(x) - \nabla_x \log Z_\theta = -\nabla_x f_\theta(x)$$

In [1]:
import os

os.environ["WANDB_NOTEBOOK_NAME"] = "score_matching_model.ipynb"

import wandb

wandb.login()
# wandb.login(relogin=True)


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from torch_geometric.nn import RotatE
from torch_geometric.datasets import FB15k_237
import sys
import os
import os.path as osp
import math
import numpy as np
from icecream import ic
import yaml
from datetime import datetime


dataset_name = "FB15k_237"

parent_dir = os.path.abspath(os.pardir)
sys.path.append(parent_dir)
data_path = osp.join(parent_dir, "data", dataset_name)

device = "cuda" if torch.cuda.is_available() else "cpu"


train_data = FB15k_237(data_path, split="train")[0].to(device)
val_data = FB15k_237(data_path, split="val")[0].to(device)
test_data = FB15k_237(data_path, split="test")[0].to(device)

wandb: Currently logged in as: bthill1 (uiuc_idealab_2024). Use `wandb login --relogin` to force relogin
/home/blaineh2/anaconda3/envs/ScoreMatchingDiffKG/lib/python3.9/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
run_sweep = True
if run_sweep:
    with open("sweep_config.yaml", "r") as file:
        sweep_config = yaml.safe_load(file)
else:
    config = {
        "batch_size": 64,
        "epochs": 100,
        "num_steps": 20,  # Adjust based on your desired SDE iterations
        "lr": 1e-4,
        "device": device,
        "k": 10,
        "similarity_metric": "cosine",
    }

In [3]:
class ScoreModel(nn.Module):
    def __init__(self, embedding_model, node_emb_dim, rel_emb_dim, config):
        super(ScoreModel, self).__init__()
        self.config = config  # store config for hyperparameters
        self.embedding_model = embedding_model
        self.node_emb_dim = node_emb_dim
        self.rel_emb_dim = rel_emb_dim
        self.score_net = nn.Sequential(
            nn.Linear(node_emb_dim + rel_emb_dim, 512, dtype=torch.float),
            nn.ReLU(),
            nn.Linear(512, 512, dtype=torch.float),
            nn.ReLU(),
            nn.Linear(512, 1, dtype=torch.float),
        )

    def forward(self, h_emb, r_emb, t_emb, timestep=None):
        # Implement your desired distance measure here (e.g., L2 distance)
        distance = torch.linalg.norm(h_emb + r_emb - t_emb, dim=-1)

        # Gradually increase the weight of the distance term during SDE steps
        weight = 0.0  # No weight increase if timestep is None
        if timestep is not None:

            def sigmoid(x):
                return 1 / (1 + math.exp(-x))

            weight = sigmoid(timestep / (self.config["num_steps"] - 1))
        score = weight * distance
        return score


def build_model(config):
    embedding_model = RotatE(
        num_nodes=train_data.num_nodes,
        num_relations=train_data.num_edge_types,
        hidden_channels=50,
    ).to(device)
    embedding_model.load_state_dict(
        torch.load("../embedding_model/FB15k_237_RotatE_embedding_model_weights.pth")
    )
    embedding_model.eval()

    # Get entity and relation embeddings
    with torch.no_grad():
        entity_embeddings_real = embedding_model.node_emb.weight.detach()
        entity_embeddings_im = embedding_model.node_emb_im.weight.detach()
        entity_embeddings = torch.cat([entity_embeddings_real, entity_embeddings_im], dim=-1)
        relation_embeddings = embedding_model.rel_emb.weight.detach()

    score_model = ScoreModel(
        embedding_model, entity_embeddings.shape[-1], relation_embeddings.shape[-1], config
    ).to(device)

    # Create data loaders
    train_dataset = TensorDataset(
        train_data.edge_index[0], train_data.edge_type, train_data.edge_index[1]
    )
    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)

    val_dataset = TensorDataset(
        val_data.edge_index[0], val_data.edge_type, val_data.edge_index[1]
    )
    val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False)

    test_dataset = TensorDataset(
        test_data.edge_index[0], test_data.edge_type, test_data.edge_index[1]
    )
    test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)

    optimizer = Adam(score_model.parameters(), lr=config["lr"])

    return score_model, optimizer, train_loader, val_loader, test_loader


def denoising_score_matching_loss(score_model, h, r, t, timestep):
    """
    Denoising score-matching loss with noise-conditional score networks.
    """
    h_emb, r_emb, t_emb = (
        score_model.embedding_model.node_emb(h),
        score_model.embedding_model.rel_emb(r),
        score_model.embedding_model.node_emb(t),
    )
    true_score = score_model(h_emb, r_emb, t_emb)  # simply do not pass in the timestep
    noisy_score = score_model(h_emb, r_emb, t_emb, timestep)
    return ((true_score - noisy_score) ** 2).mean()


def train_model(score_model, optimizer, train_loader, config):
    score_model.train()
    loss_epoch = 0
    for batch in train_loader:
        h, r, t = batch
        h, r, t = h.to(device), r.to(device), t.to(device)

        optimizer.zero_grad()
        for timestep in range(config["num_steps"]):
            loss = denoising_score_matching_loss(score_model, h, r, t, timestep)
            loss.backward()
        optimizer.step()

        loss_epoch += loss.item()

    return loss_epoch / len(train_loader)


def test_model(score_model, test_loader, config):
    score_model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            h, r, t = batch
            h, r, t = h.to(device), r.to(device), t.to(device)
            for timestep in range(config["num_steps"]):
                loss = denoising_score_matching_loss(score_model, h, r, t, timestep)
                test_loss += loss.item()

    return test_loss / (len(test_loader) * config["num_steps"])  # Average across steps


def reverse_sde(h_emb, t_emb, score_model, config):
    """
    Refine the relation embedding using reverse SDE for better link prediction.
    h_emb: Embedding of the head entity.
    t_emb: Embedding of the tail entity.
    score_model: The trained score model.
    config: Configuration dictionary.
    """
    # Initialize with random noise
    r_emb = torch.randn(score_model.rel_emb_dim, device=device)

    # Define the time steps for the reverse SDE process
    t_steps = torch.linspace(config["num_steps"] - 1, 0, config["num_steps"]).to(device)

    # Define the SDE function for the reverse process
    def sde_func(t, r_emb):
        with torch.enable_grad():
            r_emb.requires_grad_(True)
            score = score_model(h_emb, r_emb, t_emb, t)
            grad_r_emb = torch.autograd.grad(score.sum(), r_emb, create_graph=True)[0]
        return -grad_r_emb  # Reverse direction

    # Perform the reverse SDE integration
    with torch.no_grad():
        for t in reversed(t_steps):
            r_emb = sde_func(t, r_emb)

    return r_emb


def compute_metrics(score_model, test_loader, config):
    all_ranks = []
    all_hits_at_k = []

    # Get all relation embeddings from the model
    all_relations = score_model.embedding_model.rel_emb.weight.detach()

    for batch in test_loader:
        h, true_r, t = batch
        h, true_r, t = h.to(device), true_r.to(device), t.to(device)

        h_emb = score_model.embedding_model.node_emb(h)
        t_emb = score_model.embedding_model.node_emb(t)
        # Assuming reverse_sde function returns refined relation embeddings
        refined_r_emb = reverse_sde(h_emb, t_emb, score_model, config)
        if config["similarity_metric"] == "cosine":
            # Calculate cosine similarity
            similarity = F.cosine_similarity(refined_r_emb.unsqueeze(0), all_relations, dim=1)
            # Since cosine similarity is higher for closer vectors, we use largest=True
            values, indices = similarity.topk(config["k"], largest=True)
        elif config["similarity_metric"] == "l2":  # Euclidean distance
            # Expand dimensions to broadcast and calculate pairwise Euclidean distance
            dist = torch.norm(all_relations - refined_r_emb.unsqueeze(0), dim=2, p=2)

            # For Euclidean distance, closer vectors have smaller distances, so largest=False
            values, indices = dist.topk(config["k"], largest=False)

        # Calculate metrics
        # Check if true relation is within the top K predictions
        hits_at_k = (indices == true_r.unsqueeze(1)).any(dim=1).float().mean().item()
        all_hits_at_k.append(hits_at_k)

        # Calculate rank of the true relation
        for i in range(indices.size(0)):
            true_relation_idx = (indices[i] == true_r[i]).nonzero(as_tuple=True)[0]
            rank = true_relation_idx.item() + 1 if true_relation_idx.numel() > 0 else 0
            all_ranks.append(rank)

    mean_rank = np.mean(all_ranks)
    mrr = np.mean([1.0 / rank for rank in all_ranks if rank > 0])
    hits_at_k = np.mean(all_hits_at_k)

    return mean_rank, mrr, hits_at_k


def main(config=None, verbose=False):
    with wandb.init(
        project=f"ScoreMatchingDiffKG_Embedding",
        name=f"{dataset_name}_score_matching_model {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        config=config if config is not None else {},
    ):
        config = wandb.config
        model, optimizer, train_loader, val_loader, test_loader = build_model(config)
        wandb.watch(model)
        for epoch in range(config.epochs):
            loss = train_model(model, optimizer, train_loader, config)
            if verbose:
                print(f"Epoch: {epoch:03d}, Train Loss: {loss:.4f}")
            train_metrics = {"train_epoch": epoch, "train_loss": loss}
            if epoch % 10 == 0 and epoch > 0:
                loss = test_model(model, test_loader, config)
                mean_rank, mrr, hits_at_k = compute_metrics(model, test_loader, config)
                if verbose:
                    print(
                        f"Test Mean Rank: {mean_rank:.2f}, Test Mean Reciprocal Rank: {mrr:.2f}, Test Hits@K: {hits_at_k:.4f}"
                    )
                test_metrics = {
                    "test_loss": loss,
                    "test_mean_rank": mean_rank,
                    "test_mrr": mrr,
                    "test_hits_at_k": hits_at_k,
                }
            # log to wandb
            wandb.log(
                {**train_metrics, **test_metrics}
                if "test_metrics" in locals()
                else {**train_metrics}
            )

        # Save the trained model
        path = osp.join(os.getcwd(), f"{dataset_name}_score_matching_model_weights.pth")
        torch.save(model.state_dict(), path)

        # Fetch a batch from train_loader
        for batch in train_loader:
            # Assuming batch contains head_index, rel_type, tail_index, and possibly other data
            head_index, rel_type, tail_index = batch
            break  # Only need one batch for this purpose

        # Use the fetched batch to provide dummy inputs for the export
        # Ensure these variables are moved to the same device as model if necessary
        torch.onnx.export(
            model,
            (head_index, rel_type, tail_index),  # Use actual data as dummy inputs
            f"{dataset_name}_score_matching_model_weights.onnx",
            opset_version=11,
            do_constant_folding=True,
            input_names=[
                "head_index",
                "rel_type",
                "tail_index",
            ],  # Adjust input names as needed
            dynamic_axes={
                "head_index": {0: "batch_size"},
                "rel_type": {0: "batch_size"},
                "tail_index": {0: "batch_size"},
            },
        )
        wandb.save(f"{dataset_name}_score_matching_model_weights.onnx")

        return model

In [4]:
if run_sweep:

    with open("sweep_config.yaml", "r") as file:
        sweep_config = yaml.safe_load(file)

    sweep_id = wandb.sweep(project=f"ScoreMatchingDiffKG_Score_Sweep", sweep=sweep_config)

    wandb.agent(sweep_id, function=main)
else:
    model = main(config, verbose=True)

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: afxexxqn
Sweep URL: https://wandb.ai/uiuc_idealab_2024/ScoreMatchingDiffKG_Score_Sweep/sweeps/afxexxqn


wandb: Agent Starting Run: 3ahyxk10 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 400
wandb: 	k: 10
wandb: 	lr: 1.005338044831327
wandb: 	num_steps: 30
wandb: 	rel_emb_dim: 150
wandb: 	similarity_metric: l2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
